# Selection Bias: Recidivism Data

Equality:
Train of portion of data with all races

Test on data for white people
Test on data for poc

In [1]:
from conx import Network
import random

In [2]:
def getData(file):
    #open file
    tf = open(file, "r")
    content = tf.readlines()
    row_count = sum(1 for row in content)
    print(row_count)
    #Parse file
    input = []
    content = [x.strip() for x in content[1:]]
    for y in content:
        row = [col for col in y.split(",")]
        #print(row)
        input.append(row)
    tf.close()
    return input

cols = getData("compas-scores-two-years.csv")

7215


In [3]:
for c in cols:
    if len(c) == 54:
        c.remove(c[23])
        
print(len(cols[4062]))

53


In [4]:
def parseInputs(inputs):
    new = []
    row = []
    countWhat = 0
    for r in inputs:
        if r[9] == "Caucasian" or r[9] == "African-American":
            #0
            if r[5] == "Male":
                row.append(1)
            elif r[5] == "Female":
                row.append(0)
            else:
                print("error parsing col 5, Error occurs on row " + str(countWhat))
            #Age: Greater than 45, 25 - 45, and Less than 25
            #1, 2, and 3
            if r[8] == "Greater than 45":
                row.append(1)
                row.append(0)
                row.append(0)
            elif r[8] == "25 - 45":
                row.append(0)
                row.append(1)
                row.append(0)
            elif r[8] == "Less than 25":
                row.append(0)
                row.append(0)
                row.append(1)
            else:
                print("error parsing col 8, Error occurs on row " + str(countWhat))
            #race = Caucasian, African-American
            if r[9] == "Caucasian":
                row.append(0)
            elif r[9] == "African-American":
                row.append(1)
            else:
                print("error parsing col 9, Error occurs on row " + str(countWhat))
            
            #Risk of violence Score: Low, Medium, and High
            if r[44] == "Low":
                row.append(1)
                row.append(0)
                row.append(0)
            elif r[44] == "Medium":
                row.append(0)
                row.append(1)
                row.append(0)
            elif r[44] == "High":
                row.append(0)
                row.append(0)
                row.append(1)
            else:
                print("error parsing col 44, Error occurs on row " + str(countWhat))
                
            #Risk recidivism Score: Low, Medium, and High
            if r[40] == "Low":
                row.append(0)
            elif r[40] == "Medium":
                row.append(.5)
            elif r[40] == "High":
                row.append(1)
            else:
                print("error parsing col 40, Error occurs on row " + str(countWhat))
            
            #parse error in row 4754, disposed of here
            if r[40] == "":
                r = []

            if row != [] and len(row) == 9:
                new.append(row)
            #reset
            row = []
        countWhat += 1
    return new

tempinputs = parseInputs(cols)  
tempinputs[0]

error parsing col 40, Error occurs on row 4754


[1, 0, 1, 0, 1, 1, 0, 0, 0]

In [5]:
#Ensure inputs are coorect length
#Input how many rows to be processed
inputs = []
#lengthInput = int(.8 * len(tempinputs))
lengthInput = 4000
print("Total length of inputs: " + str(lengthInput))

for r in range(lengthInput):
    inputs.append(tempinputs[r])
    
inputs = [[x[0:8], x[8:]] for x in inputs]

Total length of inputs: 4000


In [6]:
net = Network(8, 60, 1)

#desired output/target
def des(inputs):
    re = int(inputs[9])
    return [re]

In [7]:
net.set_inputs(inputs)

In [8]:
net.reset()
#net.train(max_training_epochs=1000, momentum=1, epsilon=1)
net.train(max_training_epochs=1500, report_rate=250, tolerance = .25)

--------------------------------------------------
Training for max trails: 1500 ...
Epoch: 0 TSS error: 1382.07104177 %correct: 0.22325
Epoch: 250 TSS error: 342.862962276 %correct: 0.655
Epoch: 500 TSS error: 341.589637136 %correct: 0.6555
Epoch: 750 TSS error: 341.328478394 %correct: 0.65525
Epoch: 1000 TSS error: 341.154574558 %correct: 0.65525
Epoch: 1250 TSS error: 341.235575588 %correct: 0.6465
Epoch: 1500 TSS error: 341.294673608 %correct: 0.65125
--------------------------------------------------
Epoch: 1500 TSS error: 341.294673608 %correct: 0.65125


In [9]:
net.test()

--------------------------------------------------
Test:
******************************
Input : [1, 0, 1, 0, 1, 1, 0, 0]
Output: [ 0.30085907] 
Target: [1] Incorrect
******************************
Input : [0, 0, 1, 0, 0, 1, 0, 0]
Output: [ 0.25595397] 
Target: [0] Incorrect
******************************
Input : [1, 0, 0, 1, 1, 0, 0, 1]
Output: [ 0.93522866] 
Target: [1] Correct
******************************
Input : [1, 0, 1, 0, 1, 1, 0, 0]
Output: [ 0.30085907] 
Target: [0] Incorrect
******************************
Input : [1, 0, 1, 0, 0, 0, 1, 0]
Output: [ 0.68955811] 
Target: [1] Incorrect
******************************
Input : [1, 0, 1, 0, 1, 0, 0, 1]
Output: [ 0.96396292] 
Target: [1] Correct
******************************
Input : [1, 0, 1, 0, 1, 1, 0, 0]
Output: [ 0.30085907] 
Target: [0.5] Correct
******************************
Input : [1, 1, 0, 0, 0, 1, 0, 0]
Output: [ 0.0918706] 
Target: [0] Correct
******************************
Input : [1, 0, 1, 0, 1, 1, 0, 0]
Output: [ 0.30

In [10]:
wresults = []
bresults = []
wtest = []
btest = []

for p in range(len(tempinputs) - 4000):
    val = tempinputs[p+4000]
    print(val)
    #White
    if val[4] == 0:
        wtest.append(val[0:8])
    #black
    if val[4] == 1:
        btest.append(val[0:8])
    
for l in wtest:
    wresults.append(net.propagate(l))
    
for m in btest:
    bresults.append(net.propagate(m))

wFalseCount = 0
bFalseCount = 0
wMidCount = 0
bMidCount = 0
wTrueCount = 0
bTrueCount = 0
falseCountTrainData = 0
midCountTrainData = 0
trueCountTrainData = 0

for wr in wresults:
    if wr < .25:
        wFalseCount += 1

for br in bresults:
    if br < .25:
        bFalseCount += 1
        
for wr in wresults:
    if .25 <= wr <= .75:
        wMidCount += 1

for br in bresults:
    if .25 <= br <= .75:
        bMidCount += 1
        
for wr in wresults:
    if wr > .75:
        wTrueCount += 1

for br in bresults:
    if br > .75:
        bTrueCount += 1
        
for rowz in tempinputs:
    if rowz[8:] == [0]:
        falseCountTrainData += 1
        
for rowz in tempinputs:
    if rowz[8:] == [.5]:
        midCountTrainData += 1
        
for rowz in tempinputs:
    if rowz[8:] == [1]:
        trueCountTrainData += 1

print("\n")  
print("wFalseCount")   
print(wFalseCount)

print("\n")  
print("bFalseCount")   
print(bFalseCount)
        
print("falseCountTrainData")   
print(falseCountTrainData)

print("midCountTrainData")   
print(midCountTrainData)

print("trueCountTrainData")   
print(trueCountTrainData)

[1, 0, 1, 0, 1, 0, 1, 0, 1]
[1, 0, 0, 1, 1, 0, 1, 0, 0]
[1, 1, 0, 0, 0, 1, 0, 0, 0]
[1, 1, 0, 0, 1, 1, 0, 0, 0]
[1, 0, 1, 0, 0, 1, 0, 0, 0.5]
[1, 0, 0, 1, 1, 0, 1, 0, 0.5]
[1, 1, 0, 0, 1, 0, 0, 1, 0.5]
[1, 0, 1, 0, 1, 1, 0, 0, 0]
[1, 1, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 1, 1, 0, 0, 0.5]
[0, 0, 1, 0, 1, 1, 0, 0, 0.5]
[1, 0, 0, 1, 0, 0, 1, 0, 0.5]
[1, 0, 1, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 1, 0, 1, 0, 1]
[1, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 1, 0, 0.5]
[1, 0, 1, 0, 1, 1, 0, 0, 1]
[1, 0, 1, 0, 1, 0, 0, 1, 1]
[0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 1, 1, 0, 0, 1]
[1, 0, 1, 0, 1, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0]
[1, 0, 1, 0, 1, 0, 1, 0, 1]
[1, 1, 0, 0, 1, 1, 0, 0, 0]
[1, 0, 1, 0, 0, 1, 0, 0, 0.5]
[1, 0, 0, 1, 1, 0, 1, 0, 0.5]
[1, 0, 1, 0, 1, 1, 0, 0, 0.5]
[1, 0, 0, 1, 1, 0, 1, 0, 0.5]
[1, 0, 1, 0, 1, 0, 0, 1, 1]
[1, 1, 0, 0, 1, 1, 0, 0, 0]
[1, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 1, 0, 0, 0, 1, 0, 1]
[1, 0, 1, 0, 1, 0, 0, 1, 1]
[1, 0, 1, 0, 1, 1, 0, 0, 0.5]
[0, 0, 1, 0, 1, 1, 0, 0,

In [11]:
def getBias(falseCount, testSet):
    if len(testSet) == 0:
        return 0
    numer = falseCount/len(testSet)
    demo = falseCountTrainData/lengthInput
    print("Bias ratio given test dataset: ")
    return numer/demo

In [12]:
print("White cases: ")
print(getBias(wFalseCount, wtest))

White cases: 
Bias ratio given test dataset: 
0.8282299456511882


In [13]:
print("Black Cases: ")
print(getBias(bFalseCount, btest))

Black Cases: 
Bias ratio given test dataset: 
0.0788095618656292


In [14]:
print("White cases: ")
print(getBias(wMidCount, wtest))

White cases: 
Bias ratio given test dataset: 
0.3702852859206955


In [15]:
print("black cases: ")
print(getBias(bMidCount, btest))

black cases: 
Bias ratio given test dataset: 
0.8048426505527383


In [16]:
print("White cases: ")
print(getBias(wTrueCount, wtest))

White cases: 
Bias ratio given test dataset: 
0.08312526826791122


In [17]:
print("black cases: ")
print(getBias(bTrueCount, btest))

black cases: 
Bias ratio given test dataset: 
0.39798828742142744


In [27]:
wresults = []
bresults = []
wtest = []
btest = []

for p in range(len(tempinputs) - 4000):
    val = tempinputs[p+4000]
    #White
    val[4] = 0
    wtest.append(val[0:8])
    #black
    val[4] = 1
    btest.append(val[0:8])
    
for l in wtest:
    wresults.append(net.propagate(l))
    
for m in btest:
    bresults.append(net.propagate(m))

wFalseCount = 0
bFalseCount = 0
wMidCount = 0
bMidCount = 0
wTrueCount = 0
bTrueCount = 0
falseCountTrainData = 0
midCountTrainData = 0
trueCountTrainData = 0

for wr in wresults:
    if wr < .25:
        wFalseCount += 1

for br in bresults:
    if br < .25:
        bFalseCount += 1
        
for wr in wresults:
    if .25 <= wr <= .75:
        wMidCount += 1

for br in bresults:
    if .25 <= br <= .75:
        bMidCount += 1
        
for wr in wresults:
    if wr > .75:
        wTrueCount += 1

for br in bresults:
    if br > .75:
        bTrueCount += 1
        
for rowz in tempinputs:
    if rowz[8:] == [0]:
        falseCountTrainData += 1
        
for rowz in tempinputs:
    if rowz[8:] == [.5]:
        midCountTrainData += 1
        
for rowz in tempinputs:
    if rowz[8:] == [1]:
        trueCountTrainData += 1

print("\n")  
print("wFalseCount")   
print(wFalseCount)

print("\n")  
print("bFalseCount")   
print(bFalseCount)
        
print("falseCountTrainData")   
print(falseCountTrainData)

print("midCountTrainData")   
print(midCountTrainData)

print("trueCountTrainData")   
print(trueCountTrainData)



wFalseCount
1127


bFalseCount
174
falseCountTrainData
3121
midCountTrainData
1727
trueCountTrainData
1301


In [28]:
print("White cases: ")
print(getBias(wFalseCount, wtest))

White cases: 
Bias ratio given test dataset: 
0.6721306855837361


In [29]:
print("Black Cases: ")
print(getBias(bFalseCount, btest))

Black Cases: 
Bias ratio given test dataset: 
0.10377172962872236


In [30]:
print("White cases: ")
print(getBias(wMidCount, wtest))

White cases: 
Bias ratio given test dataset: 
0.457430555317414


In [31]:
print("Black Cases: ")
print(getBias(bMidCount, btest))

Black Cases: 
Bias ratio given test dataset: 
0.8683427490771248


In [32]:
print("White cases: ")
print(getBias(wTrueCount, wtest))

White cases: 
Bias ratio given test dataset: 
0.15207925893864482


In [33]:
print("Black Cases: ")
print(getBias(bTrueCount, btest))

Black Cases: 
Bias ratio given test dataset: 
0.3095260211339477


In [34]:
wresults

[array([ 0.68955811]),
 array([ 0.44671855]),
 array([ 0.0918706]),
 array([ 0.0918706]),
 array([ 0.19658718]),
 array([ 0.44671855]),
 array([ 0.63600333]),
 array([ 0.19658718]),
 array([ 0.0918706]),
 array([ 0.25595397]),
 array([ 0.25595397]),
 array([ 0.44671855]),
 array([ 0.19658718]),
 array([ 0.6413881]),
 array([ 0.19658718]),
 array([ 0.68955811]),
 array([ 0.19658718]),
 array([ 0.92816471]),
 array([ 0.25595397]),
 array([ 0.19658718]),
 array([ 0.19658718]),
 array([ 0.44671855]),
 array([ 0.68955811]),
 array([ 0.0918706]),
 array([ 0.19658718]),
 array([ 0.44671855]),
 array([ 0.19658718]),
 array([ 0.44671855]),
 array([ 0.92816471]),
 array([ 0.0918706]),
 array([ 0.19658718]),
 array([ 0.68955811]),
 array([ 0.92816471]),
 array([ 0.19658718]),
 array([ 0.25595397]),
 array([ 0.68955811]),
 array([ 0.19658718]),
 array([ 0.19658718]),
 array([ 0.68955811]),
 array([ 0.25595397]),
 array([ 0.0918706]),
 array([ 0.1997745]),
 array([ 0.68955811]),
 array([ 0.19658718

In [35]:
bresults

[array([ 0.80120534]),
 array([ 0.5332047]),
 array([ 0.29159054]),
 array([ 0.29159054]),
 array([ 0.30085907]),
 array([ 0.5332047]),
 array([ 0.97211905]),
 array([ 0.30085907]),
 array([ 0.29159054]),
 array([ 0.25164416]),
 array([ 0.25164416]),
 array([ 0.5332047]),
 array([ 0.30085907]),
 array([ 0.62424141]),
 array([ 0.30085907]),
 array([ 0.80120534]),
 array([ 0.30085907]),
 array([ 0.96396292]),
 array([ 0.25164416]),
 array([ 0.30085907]),
 array([ 0.30085907]),
 array([ 0.5332047]),
 array([ 0.80120534]),
 array([ 0.29159054]),
 array([ 0.30085907]),
 array([ 0.5332047]),
 array([ 0.30085907]),
 array([ 0.5332047]),
 array([ 0.96396292]),
 array([ 0.29159054]),
 array([ 0.30085907]),
 array([ 0.80120534]),
 array([ 0.96396292]),
 array([ 0.30085907]),
 array([ 0.25164416]),
 array([ 0.80120534]),
 array([ 0.30085907]),
 array([ 0.30085907]),
 array([ 0.80120534]),
 array([ 0.25164416]),
 array([ 0.29159054]),
 array([ 0.13481635]),
 array([ 0.80120534]),
 array([ 0.300859

In [38]:
len(bresults)

2149